# **SPRINT 4.** Projecte individual- Recol·lecció de dades

### ***Tasca 4.1***
- Objectius: Extreure dades d'una API i mitjançant Web Scraping a Python.
- Lliurament: Utilitza un fitxer de Jupyter Notebook amb l'script a Python. Recorda utilitzar Markdown per millorar la comprensió del codi. Puja'l al teu repositori de GitHub i insereix l'URL per a la seva avaluació. Recorda que el repositori ha d'estar públic.

#### Exercici 1. Consumir una API
Selecciona una API pública i extreu dades utilitzant Python.

Pasos seguidos:
1. Crear un email temporal para solicitar la API_KEY ya que el Jupyter notebook quedará como público
2. Solicitar la API_KEY en la web https://openweathermap.org/price para acceso gratuito con el email temporal.
3. Esperar 2 horas a que se active la API_KEY (en realidad tardó mas)
4. Consumir la API según la documentación de la web: https://openweathermap.org/api
5. Tener en cuenta que estamos con un cuenta gratuita, por lo que tanto los datos como el número de accesos son limitados
6. Como ejemplo, se presenta la meteorologia actual en las capitales catalanas


In [5]:
# Datos del tiempo en las capitales catalanas
# Consumimos la API para cada una de las ciudades de la lista 'cities'
# Cada una de la respuestas es un diccionario, que se almacena secuencialmente en la lista 'weather'
# Presentamos los datos básicos del tiempo, comprobando que la consulta a la API fue correcta ('.cod' == 200) 

# Se probó tambien con latitud/longitud, pero finalmente se utilizaron los nombres de las ciudades
# https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API key}

import requests

API_key = "622fbdfe465131940702e4e9b3c12ddf"
cities = ["Barcelona", "Girona", "Tarragona", "Lleida"]
weather = []

for city in cities:
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_key}"
    response = requests.get(url)
    weather.append(response.json())

print("El tiempo en las capitales catalanas:")
for w in weather:
    if w['cod'] == 200:
        print(f"  {w['name']} ({w['sys']['country']}): T: {w['main']['temp'] - 273.15:.1f} ºC, P: {w['main']['pressure']} hPa, HR: {w['main']['humidity']} %")
    else:
        print(f"*** Error en la solicitud ({response.text}) ***")
                 

El tiempo en las capitales catalanas:
  Barcelona (ES): T: 14.0 ºC, P: 1024 hPa, HR: 67 %
  Girona (ES): T: 10.9 ºC, P: 1023 hPa, HR: 86 %
  Tarragona (ES): T: 11.9 ºC, P: 1024 hPa, HR: 81 %
  Lleida (ES): T: 9.1 ºC, P: 1024 hPa, HR: 90 %


In [ ]:
# Celda de apoyo para comprobar la estructura del fichero json
weather[0]

{'coord': {'lon': 2.159, 'lat': 41.3888},
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03n'}],
 'base': 'stations',
 'main': {'temp': 290.71,
  'feels_like': 290.66,
  'temp_min': 289.51,
  'temp_max': 291.93,
  'pressure': 1018,
  'humidity': 82,
  'sea_level': 1018,
  'grnd_level': 1011},
 'visibility': 8000,
 'wind': {'speed': 5.14, 'deg': 230},
 'clouds': {'all': 40},
 'dt': 1732560154,
 'sys': {'type': 2,
  'id': 18549,
  'country': 'ES',
  'sunrise': 1732517519,
  'sunset': 1732551918},
 'timezone': 3600,
 'id': 3128760,
 'name': 'Barcelona',
 'cod': 200}

### Exercici 2. Obtenir dades amb Web Scraping
Selecciona un lloc web i extreu dades utilitzant la tècnica de Web Scraping a Python.

Los pasos seguidos han sido:
1. Se elije la web 'https://tarifaluzhora.es/' que presenta los precios de la luz del mercado regulado.
2. Segun la propia web, la fuente de los datos es REE (Red Electrica de España).
3. Los datos se presentan en un formato claro y amigable que permite comprobar fácilmente si el _scraping_ es correcto.
4. Se comprueba el fichero 'robots.txt' para verificar que tenemos permiso para capturar datos de la página principal.
5. Utilizamos la libreria BeautifulSoup para realizar el _scraping_ de las tarifas.
6. Se presentan los datos en pantalla y se comprueba que son correctos con la página web.
7. Se añaden los datos a un fichero CSV en formato fecha, hora0, precio0, hora1, precio1...


In [6]:
import requests
from bs4 import BeautifulSoup

URL = "https://tarifaluzhora.es/"

page = requests.get(URL)
if page.status_code != 200:
    print("Error en la lectura de la web '{URL}'. Codigo: {page.status_code}")

else:

    ''' Leer pagina web '''
    soup = BeautifulSoup(page.content, "html.parser")

    ''' Decodificar datos leidos y presentar en pantalla '''    
    titulo = soup.find("title").text
    horario = soup.find_all("span", itemprop="description")
    precio = soup.find_all("span", itemprop="price")
    print("", titulo[:-2])
    print("="*len(titulo))
    for i in range(len(horario)):
        print(" "*6, horario[i].text, "  :  ", precio[i].text.expandtabs(0).replace('\n', ' '))

    ''' Añadir datos leidos en una nueva linea en fichero CSV '''
    FILENAME = "precios_luz.csv"
    with open(FILENAME, 'a') as file:
        t = f"{titulo.split('|')[1].strip()[:-2]}"
        file.write(t)
        for i in range(len(horario)):
            h = horario[i].text
            p = precio[i].text.expandtabs(0).replace('\n', ' ')
            hp = ", " + h + ", " + p
            file.write(hp)
        file.write("\n")


 Precio de la tarifa de luz por horas HOY | 26 Noviembre 2024
       00:00 - 01:00   :   0.1718 €/kWh
       01:00 - 02:00   :   0.17 €/kWh
       02:00 - 03:00   :   0.166 €/kWh
       03:00 - 04:00   :   0.1628 €/kWh
       04:00 - 05:00   :   0.1662 €/kWh
       05:00 - 06:00   :   0.172 €/kWh
       06:00 - 07:00   :   0.175 €/kWh
       07:00 - 08:00   :   0.1786 €/kWh
       08:00 - 09:00   :   0.2133 €/kWh
       09:00 - 10:00   :   0.2003 €/kWh
       10:00 - 11:00   :   0.2439 €/kWh
       11:00 - 12:00   :   0.2404 €/kWh
       12:00 - 13:00   :   0.2377 €/kWh
       13:00 - 14:00   :   0.2369 €/kWh
       14:00 - 15:00   :   0.188 €/kWh
       15:00 - 16:00   :   0.1925 €/kWh
       16:00 - 17:00   :   0.1932 €/kWh
       17:00 - 18:00   :   0.2031 €/kWh
       18:00 - 19:00   :   0.2684 €/kWh
       19:00 - 20:00   :   0.2628 €/kWh
       20:00 - 21:00   :   0.2612 €/kWh
       21:00 - 22:00   :   0.2577 €/kWh
       22:00 - 23:00   :   0.2006 €/kWh
       23:00 - 24:00   :